# Initialize the accelerator

In [ ]:
from finn_examples import models
accel = models.cnv_w1a1_gtsrb()

In [ ]:
print("Expected input shape and datatype: %s %s" % (str(accel.ishape_normal), str(accel.idt)))
print("Expected output shape and datatype: %s %s" % (str(accel.oshape_normal), str(accel.odt)))

# Load the GTSRB dataset

In [ ]:
from os import path
import urllib
import numpy as np
dataset_local = "/tmp/traffic-signs-data.zip"
if not path.isfile(dataset_local):
    dataset_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"
    urllib.request.urlretrieve(dataset_url, dataset_local)
    ! unzip {dataset_local} -d /tmp

dataset_dict = np.load("/tmp/test.p")
testx = dataset_dict["features"]
testy = dataset_dict["labels"]

In [ ]:
gtsrb_classes = [
    '20 Km/h', 
    '30 Km/h', 
    '50 Km/h', 
    '60 Km/h', 
    '70 Km/h', 
    '80 Km/h', 
    'End 80 Km/h', 
    '100 Km/h', 
    '120 Km/h', 
    'No overtaking', 
    'No overtaking for large trucks', 
    'Priority crossroad', 
    'Priority road', 
    'Give way', 
    'Stop', 
    'No vehicles', 
    'Prohibited for vehicles with a permitted gross weight over 3.5t including their trailers, and for tractors except passenger cars and buses', 
    'No entry for vehicular traffic', 
    'Danger Ahead', 
    'Bend to left', 
    'Bend to right', 
    'Double bend (first to left)', 
    'Uneven road', 
    'Road slippery when wet or dirty', 
    'Road narrows (right)', 
    'Road works', 
    'Traffic signals', 
    'Pedestrians in road ahead', 
    'Children crossing ahead', 
    'Bicycles prohibited', 
    'Risk of snow or ice', 
    'Wild animals', 
    'End of all speed and overtaking restrictions', 
    'Turn right ahead', 
    'Turn left ahead', 
    'Ahead only', 
    'Ahead or right only', 
    'Ahead or left only', 
    'Pass by on right', 
    'Pass by on left', 
    'Roundabout', 
    'End of no-overtaking zone', 
    'End of no-overtaking zone for vehicles with a permitted gross weight over 3.5t including their trailers, and for tractors except passenger cars and buses', 
    'Not a roadsign'
]

In [ ]:
print("Dataset shape is " + str(testx.shape))

# Classify a single image

In [ ]:
test_single_x = testx[0]
test_single_y = testy[0]

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.imshow(test_single_x)
plt.show()

In [ ]:
print("Expected class is:\n%s" % (gtsrb_classes[test_single_y]))

In [ ]:
accel_y = accel.execute(test_single_x.reshape(accel.ishape_normal))
print("Accelerator result is:\n%s" % (gtsrb_classes[np.argmax(accel_y)]))

# Validate accuracy on GTSRB test set

In [ ]:
batch_size = 421
total = testx.shape[0]
accel.batch_size = batch_size
n_batches = int(total / batch_size)

batch_imgs = testx.reshape(n_batches, batch_size, -1)
batch_labels = testy.reshape(n_batches, batch_size)
obuf_normal = np.empty_like(accel.obuf_packed_device)
print("Ready to run validation, test images tensor has shape %s" % str(batch_imgs.shape))
print("Accelerator buffer shapes are %s for input, %s for output" % (str(accel.ishape_packed), str(accel.oshape_packed)) )

In [ ]:
ok = 0
nok = 0
for i in range(n_batches):
    ibuf_normal = batch_imgs[i].reshape(accel.ibuf_packed_device.shape)
    exp = batch_labels[i]
    # to avoid the slower software implementation during data unpacking,
    # we make manual calls to buffer copies and execute_on_buffers
    # all this could have been replaced with accel.execute() otherwise
    accel.copy_input_data_to_device(ibuf_normal)
    accel.execute_on_buffers()
    obuf_normal = np.empty_like(accel.obuf_packed_device)
    accel.copy_output_data_from_device(obuf_normal)
    # this line provides fast unpacking using numpy primitives
    # instead of using FINN's unpack functions
    quick_out = obuf_normal.view(np.uint16).reshape(accel.batch_size, 44)
    obuf_argmax = np.argmax(quick_out, axis=-1)
    ok_batch = (obuf_argmax == exp).sum()
    nok_batch = (batch_size-ok_batch)
    ok += ok_batch
    nok += nok_batch
    
    print("batch %d / %d : total OK %d NOK %d" % (i+1, n_batches, ok, nok))

In [ ]:
acc = 100.0 * ok / (total)
print("Final accuracy: {}%".format(acc))

# Run built-in benchmarks

In [ ]:
accel.batch_size = 100
accel.throughput_test()